In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

In [137]:
df = pd.read_csv('BenchmarkingSample20230607.csv')
df

,Unit,Period,Days,Account,Amount
0,00040302CD88,1,30,Sales,89952
1,00040302CD88,2,31,Sales,90594
2,00040302CD88,3,30,Sales,81909
3,00040302CD88,4,31,Sales,87418
4,00040302CD88,5,31,Sales,91109
...,...,...,...,...,...
6043,0004030BEB57,20,30,Other Non-controllables,1455
6044,0004030BEB57,21,31,Other Non-controllables,7456
6045,0004030BEB57,22,31,Other Non-controllables,2037
6046,0004030BEB57,23,28,Other Non-controllables,2037


## Mean Analysis

In [183]:
calculate = calculate.drop_duplicates(keep='first')
calculate

,Unit,Days,Account,Amount,Sales,Cost of Sales - Food,Cost of Sales - Paper,Crew Labor,Management Labor,Other Labor-related Costs,Other Controllables,Advertising,Promotion,Outside Services,Maintenance & Repair,Utilities,Rent,Other Non-controllables
Period,,,,,,,,,,,,,,,,,,
1,00040302CD88,30,Sales,89952,90594,20876,2058,1376,4255,2745,373,3805,445,5329,426,4917,-178,4530
2,00040302CD88,31,Sales,90594,81909,19531,2291,178,4241,2376,458,3440,553,5062,1192,4692,-527,4095
3,00040302CD88,30,Sales,81909,87418,20999,2056,36597,5515,2300,850,3672,874,5967,2156,5699,-353,4371
4,00040302CD88,31,Sales,87418,91109,21357,2312,29942,5484,2451,349,3827,635,5727,999,5109,-167,4555
5,00040302CD88,31,Sales,91109,94117,26257,3913,29863,5416,2452,437,3953,409,5692,3832,5120,-291,4706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,0004030BEB57,30,Other Non-controllables,1455,139440,39325,5077,38354,3210,2377,597,3384,843,9601,1292,6572,-4587,6972
21,0004030BEB57,31,Other Non-controllables,7456,125878,37534,3907,34158,3881,2829,275,5287,614,8059,2434,7411,-4824,6294
22,0004030BEB57,31,Other Non-controllables,2037,121580,36857,4128,33044,4405,2745,361,5106,193,7570,780,1840,-4765,6079


In [190]:
calculate = df.pivot_table(index=['Unit', 'Period'], columns='Account', values='Amount', aggfunc='sum').reset_index()

Calculate Total cost and profit based on 'calculate'

In [194]:
# Create L&P table
output = pd.DataFrame()
output['Unit'] = calculate['Unit']


output['Total Cost of Sales'] = calculate['Cost of Sales - Food'] + calculate['Cost of Sales - Paper']
output['Gross Profit'] = calculate['Sales'] - output['Total Cost of Sales']
output['Total Labor'] = calculate[['Crew Labor', 'Management Labor', 'Other Labor-related Costs']].sum(axis=1)
output['Total Prime Costs'] = output['Total Cost of Sales'] + output['Total Labor']
output['Total Controllable Costs'] =  calculate[['Crew Labor','Management Labor','Other Labor-related Costs','Advertising','Promotion','Outside Services','Maintenance & Repair','Other Controllables','Utilities']].sum(axis=1)
output['Controllable Profit'] = output['Gross Profit'] - output['Total Controllable Costs']
output['Total Non-controllable Costs'] = calculate['Rent'] + calculate['Other Non-controllables']
output['Store EBITDA'] = output['Gross Profit'] - output['Total Controllable Costs'] - output['Total Non-controllable Costs']

output

,Unit,Total Cost of Sales,Gross Profit,Total Labor,Total Prime Costs,Total Controllable Costs,Controllable Profit,Total Non-controllable Costs,Store EBITDA
0,00040302CD88,22747,67205,7898,30645,34531,32674,7269,25405
1,00040302CD88,22934,67660,9330,32264,29278,38382,7740,30642
2,00040302CD88,21822,60087,8033,29855,27747,32340,6944,25396
3,00040302CD88,23055,64363,49056,72111,71261,-6898,7184,-14082
4,00040302CD88,23669,67440,42535,66204,64950,2490,6218,-3728
...,...,...,...,...,...,...,...,...,...
211,0004030BEB57,109793,264704,94918,204711,139868,124836,91047,33789
212,0004030BEB57,121644,303041,114744,236388,156370,146671,120684,25987
213,0004030BEB57,112525,250680,103340,215865,148349,102331,86448,15883
214,0004030BEB57,110105,247506,105045,215150,141023,106483,84785,21698


In [195]:
output.to_excel('RMS_calculation.xlsx', index=False)

Mean Analysis

In [196]:
# Create a sample time series data
data = output
mean_data = pd.DataFrame(data)

# Calculate the SMA with a window size of 3
mean_data['Total Cost of Sales_SMA'] = output['Total Cost of Sales'].rolling(window=3).mean()
mean_data['Gross Profit_SMA'] = output['Gross Profit'].rolling(window=3).mean()
mean_data['Total Labor_SMA'] = output['Total Labor'].rolling(window=3).mean()
mean_data['Total Prime Costs_SMA'] = output['Total Prime Costs'].rolling(window=3).mean()
mean_data['Total Controllable Costs_SMA'] = output['Total Controllable Costs'].rolling(window=3).mean()
mean_data['Controllable Profit_SMA'] = output['Controllable Profit'].rolling(window=3).mean()
mean_data['Total Non-controllable Costs_SMA'] = output['Total Non-controllable Costs'].rolling(window=3).mean()
mean_data['Store EBITDA_SMA'] = output['Store EBITDA'].rolling(window=3).mean()


mean_data = mean_data.round(2)
mean_data

,Unit,Total Cost of Sales,Gross Profit,Total Labor,Total Prime Costs,Total Controllable Costs,Controllable Profit,Total Non-controllable Costs,Store EBITDA,Total Cost of Sales_SMA,Gross Profit_SMA,Total Labor_SMA,Total Prime Costs_SMA,Total Controllable Costs_SMA,Controllable Profit_SMA,Total Non-controllable Costs_SMA,Store EBITDA_SMA
0,00040302CD88,22747,67205,7898,30645,34531,32674,7269,25405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00040302CD88,22934,67660,9330,32264,29278,38382,7740,30642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00040302CD88,21822,60087,8033,29855,27747,32340,6944,25396,22501.00,64984.00,8420.33,30921.33,30518.67,34465.33,7317.67,27147.67
3,00040302CD88,23055,64363,49056,72111,71261,-6898,7184,-14082,22603.67,64036.67,22139.67,44743.33,42762.00,21274.67,7289.33,13985.33
4,00040302CD88,23669,67440,42535,66204,64950,2490,6218,-3728,22848.67,63963.33,33208.00,56056.67,54652.67,9310.67,6782.00,2528.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0004030BEB57,109793,264704,94918,204711,139868,124836,91047,33789,110990.67,266104.33,98210.00,209200.67,141025.67,125078.67,91994.67,33084.00
212,0004030BEB57,121644,303041,114744,236388,156370,146671,120684,25987,114948.33,281048.33,104379.67,219328.00,146175.67,134872.67,102668.33,32204.33
213,0004030BEB57,112525,250680,103340,215865,148349,102331,86448,15883,114654.00,272808.33,104334.00,218988.00,148195.67,124612.67,99393.00,25219.67
214,0004030BEB57,110105,247506,105045,215150,141023,106483,84785,21698,114758.00,267075.67,107709.67,222467.67,148580.67,118495.00,97305.67,21189.33


In [197]:
mean_data.columns

Index(['Unit', 'Total Cost of Sales', 'Gross Profit', 'Total Labor',
       'Total Prime Costs', 'Total Controllable Costs', 'Controllable Profit',
       'Total Non-controllable Costs', 'Store EBITDA',
       'Total Cost of Sales_SMA', 'Gross Profit_SMA', 'Total Labor_SMA',
       'Total Prime Costs_SMA', 'Total Controllable Costs_SMA',
       'Controllable Profit_SMA', 'Total Non-controllable Costs_SMA',
       'Store EBITDA_SMA'],
      dtype='object')

In [198]:
mean_data = mean_data[['Unit',
       'Total Cost of Sales_SMA', 'Gross Profit_SMA', 'Total Labor_SMA',
       'Total Prime Costs_SMA', 'Total Controllable Costs_SMA',
       'Controllable Profit_SMA', 'Total Non-controllable Costs_SMA',
       'Store EBITDA_SMA']]

mean_data

,Unit,Total Cost of Sales_SMA,Gross Profit_SMA,Total Labor_SMA,Total Prime Costs_SMA,Total Controllable Costs_SMA,Controllable Profit_SMA,Total Non-controllable Costs_SMA,Store EBITDA_SMA
0,00040302CD88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00040302CD88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00040302CD88,22501.00,64984.00,8420.33,30921.33,30518.67,34465.33,7317.67,27147.67
3,00040302CD88,22603.67,64036.67,22139.67,44743.33,42762.00,21274.67,7289.33,13985.33
4,00040302CD88,22848.67,63963.33,33208.00,56056.67,54652.67,9310.67,6782.00,2528.67
...,...,...,...,...,...,...,...,...,...
211,0004030BEB57,110990.67,266104.33,98210.00,209200.67,141025.67,125078.67,91994.67,33084.00
212,0004030BEB57,114948.33,281048.33,104379.67,219328.00,146175.67,134872.67,102668.33,32204.33
213,0004030BEB57,114654.00,272808.33,104334.00,218988.00,148195.67,124612.67,99393.00,25219.67
214,0004030BEB57,114758.00,267075.67,107709.67,222467.67,148580.67,118495.00,97305.67,21189.33


In [199]:
# Export mean_data 

mean_data.to_excel('RMS_SMA.xlsx', index=False)